### Fitting DCC GARCH model using R - switch kernel

In [ ]:
#install.packages("rmgarch")
suppressMessages(library(rmgarch))
suppressMessages(library(parallel))
suppressMessages(library(quantmod))
suppressMessages(library(tidyverse))
suppressMessages(library(xts))
suppressMessages(library(ggplot2))
suppressMessages(library(reshape2))

In [ ]:
etfs <- read.csv("../data/return_data.csv", sep=";")

In [ ]:
#etfs <- etfs %>% select(-Date)
etfs <- etfs %>% mutate(Date=as.Date(Date, format = "%Y-%m-%d"))

### Converting dataset to xts

In [ ]:
xts_etfs <- xts(etfs, order.by = etfs[,1])

In [ ]:
# garch models: sGARCH, eGARCH, gjrGARCH
# garch order: 1,1 - 1,0
# distribution model: std

# (sGARCH, 1,0), (sGARCH. 1, 1), (gjrGARCH, 1, 1), (eGARCH, 1, 1)

In [ ]:
(xts_etfs)

In [ ]:
xspec = ugarchspec(mean.model = list(armaOrder = c(0, 0)), variance.model = list(garchOrder = c(1,1), model = 'sGARCH'), distribution.model = 'std')
uspec = multispec(replicate(ncol(xts_etfs)-1, xspec))
spec1 = dccspec(uspec = uspec, dccOrder = c(1, 1), distribution = 'mvt')
cl = makePSOCKcluster(4)
multf = multifit(uspec, xts_etfs[,2:ncol(xts_etfs)], cluster = cl)
fit1 = dccfit(spec1, data = xts_etfs[,2:ncol(xts_etfs)], fit.control = list(eval.se = TRUE), fit = multf, cluster = cl)
print(fit1)           
stopCluster(cl)

In [ ]:
data.frame(lapply(coef(fit1), type.convert), stringsAsFactors=FALSE) %>% write.csv("garch_params.csv")

In [ ]:
last_Omega <- tail(rcov(fit1, output="matrix"), 1)

In [ ]:
tail(rcov(fit1, output="matrix"), 1) %>% write.csv('Omega_t.csv')

In [ ]:
d <- matrix(last_Omega, nrow = 4, byrow = TRUE)

In [ ]:
mat <- matrix(0, nrow = 4, ncol = 4)
mat[upper.tri(mat, diag = TRUE)] <- last_Omega

In [ ]:
mat

In [ ]:
#coef(fit1) %>% write.csv('coef.csv')
#install.packages("gmvarkit")
library(gmvarkit)

In [ ]:
etf_forecast <- dccforecast(fit1, n.ahead=1)

In [ ]:
rcov(etf_forecast, output="matrix")

In [ ]:
rcov(etf_forecast, output="matrix")

In [ ]:
etf_forecast

### Using DCC-roll to forecast

In [ ]:
cl = makePSOCKcluster(8)
roll <- dccroll(spec = spec1, data = xts_etfs[,2:11], forecast.length = 1, n.ahead=1, refit.every = 1,
               refit.window = "recursive")

#,             fit.control = list(eval.se = TRUE, stationarity = TRUE, scale = FALSE))
stopCluster(cl)

In [ ]:
data <- as_tibble(rcov(roll, output="matrix")) %>%
        mutate(Date = seq.int(1, nrow(rcov(roll, output="matrix")), 1), .before=EEM) 

print(data)

In [ ]:
data %>% ggplot(aes(x = Date)) +
geom_line(color = Series) +
# Add a second axis and specify its features
labs(
  title="Macroeconomic indicators",
  x = "Year",
  y = "Count")


#rcov(roll, output="matrix")